In [1]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn


In [3]:
def select_total_rev_less_than_1000(conn):
    """
    Q10: Produce a list of facilities with a total revenue less than 1000.
    The output of facility name and total revenue, sorted by revenue. Remember
    that there's a different cost for guests and members!
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT * 
        FROM (SELECT f.name, 
                     SUM(CASE WHEN b.memid <> 0 THEN membercost
                              ELSE guestcost *2 END) total_rev
              FROM Facilities f
              LEFT JOIN Bookings b
                     ON f.facid = b.facid
              GROUP BY f.facid
              ORDER BY total_rev) a
        WHERE total_rev < 1000
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [4]:
def recommended_by(conn):
    """
    Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT  m1.surname, 
                m1.firstname, 
                m2.firstname || ' ' ||  m2.surname AS referred_by
        FROM Members m1
             LEFT JOIN Members m2
                    ON m1.recommendedby = m2.memid
        WHERE m1.recommendedby <> 0 
        ORDER BY m1.surname, m1.firstname
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [5]:
def usage_by_member(conn):
    """
    Q12: Find the facilities with their usage by member, but not guests
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT DISTINCT f.name
        FROM Bookings b
             INNER JOIN Facilities f
                     ON b.facid = f.facid
        WHERE memid <> 0
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [14]:
def usage_by_month(conn):
    """
    Q13: Find the facilities usage by month, but not guests
    """
    cur = conn.cursor()

    query1 = """
        SELECT f.facid,
               strftime('%m', starttime) AS month,
               COUNT(bookid)
        FROM Facilities AS f
             LEFT JOIN Bookings AS b
                    ON f.facid = b.facid
        WHERE memid <> 0
        GROUP BY f.facid, month
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [15]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print()
        print("Q10. Select facilities with total revenue less than 1000")
        select_total_rev_less_than_1000(conn)
        print()
        print("Q11. A report of members and who recommended them")
        recommended_by(conn)
        print()
        print("Q12. Find the facilities with their usage by member, but not guests")
        usage_by_member(conn)
        print()
        print("Q13. Find the facilities usage by month, but not guests")
        usage_by_month(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0

Q10. Select facilities with total revenue less than 1000
('Table Tennis', 180)
('Snooker Table', 230)
('Pool Table', 530)

Q11. A report of members and who recommended them
('Bader', 'Florence', 'Ponder Stibbons')
('Baker', 'Anne', 'Ponder Stibbons')
('Baker', 'Timothy', 'Jemima Farrell')
('Boothe', 'Tim', 'Tim Rownam')
('Butters', 'Gerald', 'Darren Smith')
('Coplin', 'Joan', 'Timothy Baker')
('Crumpet', 'Erica', 'Tracy Smith')
('Dare', 'Nancy', 'Janice Joplette')
('Farrell', 'David', None)
('Farrell', 'Jemima', None)
('GUEST', 'GUEST', None)
('Genting', 'Matthew', 'Gerald Butters')
('Hunt', 'John', 'Millicent Purview')
('Jones', 'David', 'Janice Joplette')
('Jones', 'Douglas', 'David Jones')
('Joplette', 'Janice', 'Darren Smith')
('Mackenzie', 'Anna', 'Darren Smith')
('Owen', 'Charles', 'Darren Smith')
('Pinker', 'David', 'Jemima Farrell')
('Purview', 'Millicent', 'Tracy Smith')
('Rownam', 'Tim', None)
('Rumney', 'Henrietta', 'Matthew Genting')
('Sarwin', 'Ramnaresh', 'Florence